In [1]:
from rdkit import Chem

# 1. SMILES/SMARTS
SMARTS规则速查表:https://www.jianshu.com/p/ec1194c30422
1. 原子表示:
- 使用元素符号表示原子，如碳C，氧O
- 使用"[]"表示同位素，如[13C]表示碳的同位素-13
2. 键表示:
- 单键: 使用链接原子的直线"-"表示，如C-C表示碳与碳之间的单键
- 双键: 使用"="表示，如C=C表示碳与碳之间的双键
- 三键: 使用"#"表示，如C#C表示碳与碳之间的三键
- 芳香键: 使用":"表示，如C:C表示芳香环中的碳与碳之间的键
3. 环表示:
- 使用**数字**表示环的闭合，如果结构中有环，则要打开
- 断开处的两个原子用同一个数字标记，表示原子间有键相连，如环己烷(C6H12)，使用C1CCCCC1表示
- 芳环中的C, O, S, N原子分别用小写字母c, o, s, n表示
4. 支链表示:
- 使用小括号"()"表示支链，如CC(C)C表示有一个支链的丙烷
5. 立体化学表示:
- 使用"@"和"@@"表示手性，"@"表示逆时针顺序的手性中心，"@@"表示顺时针顺序的手性中心

(1) 从SMILES/SMARTS直接读取

In [2]:
smi = 'CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)NC2=O)(C)F)O)OC3=CC=CC=C3'
mol = Chem.MolFromSmiles(smi)
sms = Chem.MolFromSmarts('Cc1ccccc1')

print(mol)
print(sms)

(2) 批量读取smiles
1. 从.smi批量读取: SmileMolSupplier(data, delimiter, smilsColumn, nameColumn, titleLine, sanitize)
  - data: 数据文件
  - delimiter: 分隔符，默认为'\t'
  - silsColumn: SMILES所在列，默认为0
  - nameColumn: SMILES名称所在列，默认为1
  - titleLine: 是否含有标题行，默认为True
  - sanitize: 是否检查正确性，默认为True
  - 返回可迭代器

In [3]:
suppl = Chem.SmilesMolSupplier('data/batch_smiles.smi', delimiter='\t')
print(suppl)
mols = [Chem.MolToSmiles(mol) for mol in suppl]
print(mols)

['C1=CC=CC=CC=C1', 'c1ccccc1', 'c1ccoc1']


2. 从文本批量读取: SmilesMolSupplierFromText()
- 参数基本同上

In [4]:
with open('data/batch_smiles.smi', 'r') as f:
    mols_text = f.read()
suppl = Chem.SmilesMolSupplierFromText(mols_text, delimiter='\t')
mols = [Chem.MolToSmiles(mol) for mol in suppl]
print(mols)

['C1=CC=CC=CC=C1', 'c1ccccc1', 'c1ccoc1']


3. 读取DataFrame中的SMILES
- PandasTools.AddMoleculeColumnToFrame(frame, smilesCol, molCol, includeFingerprints)
- frame: DataFrame对象
- smilesCol: SMILES所在列
- molCol: 新列名，将存放产生的rdkit mol对象
- includeFingerprints: 是否生成指纹

In [5]:
import pandas as pd
from rdkit.Chem import PandasTools, Descriptors

df = pd.read_csv('data/smiles_df.csv')
df

,Name,SMILES
0,Lanreotide,c1(c2c(cccc2)[nH]c1)C[C@@H]1NC(=O)[C@@H](Cc2cc...
1,Lansoprazole,Cc1c(OCC(F)(F)F)ccnc1CS(=O)c2nc3ccccc3[nH]2
2,Laromustine,CS(=O)(=O)[N@@](N(S(=O)(=O)C)C(=O)NC)CCCl
3,Laropiprant,CS(=O)(=O)c1cc(cc2c1n(c3c2CC[C@@H]3CC(=O)O)Cc4...
4,Larotaxel,O=C1[C@@]23[C@@H]([C@@]4(OC(=O)C)[C@@H](C[C@@H...


In [6]:
PandasTools.AddMoleculeColumnToFrame(df, 'SMILES', 'mol', includeFingerprints=True)
df['MW'] = df['mol'].apply(Descriptors.MolWt)
print(df.head(2))

           Name                                             SMILES   
0    Lanreotide  c1(c2c(cccc2)[nH]c1)C[C@@H]1NC(=O)[C@@H](Cc2cc...  \
1  Lansoprazole        Cc1c(OCC(F)(F)F)ccnc1CS(=O)c2nc3ccccc3[nH]2   

                                                 mol        MW  
0  <rdkit.Chem.rdchem.Mol object at 0x000002B63AD...  1096.347  
1  <rdkit.Chem.rdchem.Mol object at 0x000002B63AD...   369.368  


# 2. 读入sdf文件
- 从.sdf里批量读取: Chem.SDMolSupplier(fileName, sanitize, removeHs, strictParsing)
- fileName: 文件名
- sanitize: 检查化合价，计算芳香性，共轭，杂化，kekule，默认True
- removeHs: 移除氢原子，默认True
- strictParsing: 严格解析，默认True

In [8]:
from rdkit import Chem
suppl = Chem.SDMolSupplier('data/batch.sdf')
mols = [Chem.MolToSmiles(mol) for mol in suppl if mol]
print(mols)

['C1=C\\C=C/C=C\\C=C/1', 'c1ccccc1', 'c1ccoc1']


# 3. 读入mol文件
- 从.mol里读取: Chem.MolFromMolFile(fileName, sanitize, removeHs, strictParsing)

In [10]:
m = Chem.MolFromMolFile('data/output.mol')
print(Chem.MolToSmiles(m))

C1CCC1


# 4. 读入多肽字符串

In [11]:
seq = 'GGGGG'
mol = Chem.MolFromSequence(seq)
smi = Chem.MolToSmiles(mol)
print('smi', smi)

smi NCC(=O)NCC(=O)NCC(=O)NCC(=O)NCC(=O)O


# 5. 读入inchi

In [13]:
inchi = 'InChI=1S/C6H12/c1-2-4-6-5-3-1/h1-6H2'
mol = Chem.inchi.MolFromInchi(inchi)
smi = Chem.MolToSmiles(mol)
print(smi)

C1CCCCC1
